# maude-magic

> Execute a maude`s session in Jupyter Lab

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)
- [IPython magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html)
- See persistent Shells.ipynb  

In [1]:
%%bash 
for package in pexpect nbdev jupyterlab-quarto
do  
    pip list | grep   "^$package *" > /dev/null && echo "$package found" || pip install $package
done
# This must be done in terminal
# nbdev_install_quarto

pexpect found
nbdev found
jupyterlab-quarto found


## The Maude Interpreter class

In [2]:
#| default_exp maude-magic

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
import os
import pexpect
import time

In [5]:
#| export
timeout = 3

class TimeoutException(Exception):
    """Exception raised if time-out."""

    def __init__(self):
        super().__init__("Timeout exception.")
        self.error_code = 1

    def __str__(self):
        return f"{self.message} (Error Code: {self.error_code})"    


class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        self.debug,self.timeout = debug,timeout
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
        # the expawned process:
        self.sh = pexpect.spawn('maude', encoding='utf-8', env=env)
        self.sh.expect('Maude> ')
        self._sync()    

    def _sync(self):
        """ We syncrhonize with maude shell requesting a pwd.""" 
        self.sh.sendline('pwd')
        self.sh.expect('pwd.*Maude> ')
            
    def __del__(self):
        if self.debug: print('Destroying Object')
        self.__call__('quit .')
             
    # Cmmand processing
    def __call__(self,command,timeout=timeout):
        """ Adds '\n' to command if it don't terminate with it. """
        # Don't terminate session by command
        if self.debug: print(f"Original Command = {repr(command)}")
        if command == 'quit .': return ''
        # strip command before send
        if self.debug: print(f"<--{repr(command.strip())}")
        if command[-1] != '\n' : command += '\n'
        if self.debug: print(f"Sent Command = {repr(command)}")
        self.sh.send(command)
        self._sync()
        if self.debug: print(f"-->{repr(self.sh.before)+repr(self.sh.after)}")
        response = self.sh.before
        # filter response
        return response
        

Creating maude interpreter:

In [6]:
maude=MaudeInterpreter(debug=True)

MAUDE_LIB=/usr/local/lib/Maude-3.5.1-linux-x86_64/:/home/usuario:/home/usuario/maude-magic/nbs


Load a maude module. Show it and make some reduction:

In [7]:
print(maude('load SIMPLE-NAT .'))
print(maude('show module .'))
print(maude('red s s zero .'))

Original Command = 'load SIMPLE-NAT .'
<--'load SIMPLE-NAT .'
Sent Command = 'load SIMPLE-NAT .\n'
-->'load SIMPLE-NAT .\n\r\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
load SIMPLE-NAT .
Maude> 
Original Command = 'show module .'
<--'show module .'
Sent Command = 'show module .\n'
-->'show module .\n\rfmod SIMPLE-NAT is\r\n  sort Nat .\r\n  op zero : -> Nat .\r\n  op s_ : Nat -> Nat .\r\n  op _+_ : Nat Nat -> Nat .\r\n  vars M N : Nat .\r\n  eq zero + N = N .\r\n  eq s N + M = s (N + M) .\r\nendfm\r\n\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
show module .
fmod SIMPLE-NAT is
  sort Nat .
  op zero : -> Nat .
  op s_ : Nat -> Nat .
  op _+_ : Nat Nat -> Nat .
  vars M N : Nat .
  eq zero + N = N .
  eq s N + M = s (N + M) .
endfm
Maude> 
Original Command = 'red s s zero .'
<--'red s s zero .'
Sent Command = 'red s s zero .\n'
-->'red s s zero .\n\rreduce in SIMPLE-NAT : s s zero .\r\nrewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)\r\nresult Nat: s s zero\r\n\rMaude> ''pwd\n\r/

Skip `quit` command:

In [8]:
maude('quit .')

Original Command = 'quit .'


''

Closing maude session on object destroy:

In [9]:
del maude 

Destroying Object
Original Command = 'quit .'


## The maude Magic Class

In [10]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

In [11]:
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
# from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MaudeMagics(Magics):
    """Adapts Maude Shell to a IPython Magic class.Uses a owned Maude Shell.
       Cell magics are used to execute maude commands.
       Line magics are used for command line options."""  
    def __init__(self,shell):
        # Create the owned Maude Shell instance an pass it as shell
        super(MaudeMagics,self).__init__(MaudeInterpreter(debug=False)) 
        # print("On Construntor:"+str(type(self.shell)))
        self.line_counter=0
        
    def prepare_request(self,cell_contents:str)->str:
        """ - Removes leading and trailing empty lines from cell
            - Removes leading and trailing empty lines from each line
            - Terminates each line with \r\n
            - Count efective lines sent to maude shell.
        """
        # print(f"cell_contents at maude()={cell_contents}")
        cell_contents=cell_contents.strip()
        request = ""
        for cell_line in cell_contents.split('\n'):
            request+=cell_line.strip()+'\n'
            self.line_counter+=1
        # Maude shell will add trailing \n
        return request
    
    __prepare_request=prepare_request 

    def prepare_response(self,shell_response:str)->str:
        """ Ads the count of sent lines at the header of respone,
            to ease sintax error location.
        """    
        response =  f"{self.line_counter} (lines sent before.)\n"
        response += ("--------------------------------------------\n")
        response += shell_response + '\n'
        return response
    
    __prepare_response = prepare_response     
           
    @line_cell_magic
    def maude(self, line, cell=None):
        if cell is None:
            return self.shell(line)
        else:
            # print(f"cell at maude()={cell}")
            print(self.prepare_response(self.shell(self.prepare_request(cell))))

# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MaudeMagics)

Manually executing 'load_ipython_extension' for test purposes:

In [12]:
load_ipython_extension(get_ipython())

Now, MaudeMagic uses an owned MaudeInterpreter to run maude commands:

In [13]:
result = %maude load SIMPLE-NAT . 
result
    #assert result == 'Maude>'

'load SIMPLE-NAT .\n\r\rMaude> '

In [14]:
%%maude
show module .  

1 (lines sent before.)
--------------------------------------------
show module .
fmod SIMPLE-NAT is
  sort Nat .
  op zero : -> Nat .
  op s_ : Nat -> Nat .
  op _+_ : Nat Nat -> Nat .
  vars M N : Nat .
  eq zero + N = N .
  eq s N + M = s (N + M) .
endfm
Maude> 



In [15]:
%%maude
red s s zero .    

2 (lines sent before.)
--------------------------------------------
red s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude> 



## Use cases as test

### JSON Syntax

[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/)

In [16]:
%%maude 
fmod JSON-SORTS is
    sort Json 
    sorts  Map List .
    subsorts Map List < Json .    
    sort Entry .    
    sorts Key Value .
    op _:_ : Key Value  -> Entry [ctor] .
endfm         
  
view Json  from TRIV to JSON-SORTS is sort Elt to Json . endv
view Entry from TRIV to JSON-SORTS is sort Elt to Entry . endv

fmod JSON-STRUCT is
    protecting LIST{Json} * ( op (__) to (_,_) ) .
    protecting SET{Entry} .
    op [_] : List{Json} -> List .
    op {_} : Set{Entry} -> Map .
endfm        

fmod JSON is 
    protecting JSON-STRUCT .
    protecting STRING .

    subsort String Map List < Key Value Json .
        
endfm 

28 (lines sent before.)
--------------------------------------------
fmod JSON-SORTS is
> sort Json
> sorts  Map List .
> subsorts Map List < Json .
> sort Entry .
> sorts Key Value .
> op _:_ : Key Value  -> Entry [ctor] .
> endfm
Maude> 
> view Json  from TRIV to JSON-SORTS is sort Elt to Json . endv
Maude> view Entry from TRIV to JSON-SORTS is sort Elt to Entry . endv
Maude> 
> fmod JSON-STRUCT is
> protecting LIST{Json} * ( op (__) to (_,_) ) .
> protecting SET{Entry} .
> op [_] : List{Json} -> List .
> op {_} : Set{Entry} -> Map .
> endfm
Maude> 
> fmod JSON is
> protecting JSON-STRUCT .
> protecting STRING .
> 
> subsort String Map List < Key Value Json .
> 
> endfm
Maude> 



In [17]:
result = %maude red "Juan" : "Perico" .
print(result)    
assert 'Entry: "Juan" : "Perico"' in result      

red "Juan" : "Perico" .
reduce in JSON : "Juan" : "Perico" .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Entry: "Juan" : "Perico"
Maude> 


In [18]:
result = %maude red \
{\
  "name" : "Manu Sporny" ,\
  "homepage" : "http://manu.sporny.org/" ,\
  "image" : "http://manu.sporny.org/images/manu.png"\
} .
assert "Map:" in result    

In [19]:
result = %maude red ["Juan","Perico","Andres"] .
print(result)    
assert 'List: ["Juan", "Perico", "Andres"]' in result


red ["Juan","Perico","Andres"] .
reduce in JSON : ["Juan", "Perico", "Andres"] .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result List: ["Juan", "Perico", "Andres"]
Maude> 


In [20]:
result = %maude red ["Juan",["Perico","Andres"]] .
print(result)    
assert 'List: ["Juan", ["Perico", "Andres"]]' in result 

red ["Juan",["Perico","Andres"]] .
reduce in JSON : ["Juan", ["Perico", "Andres"]] .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result List: ["Juan", ["Perico", "Andres"]]
Maude> 


In [21]:
result = %maude red ["Juan", {"Perico" : "Andres"} ] .
print(result)    
assert 'List: ["Juan", {"Perico" : "Andres"}]' in result 

red ["Juan", {"Perico" : "Andres"} ] .
reduce in JSON : ["Juan", {"Perico" : "Andres"}] .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result List: ["Juan", {"Perico" : "Andres"}]
Maude> 


In [22]:
result = %maude red {"Juan" : {"Perico" : "Andres"} } .
print(result)    
assert 'Map: {"Juan" : {"Perico" : "Andres"}}' in result 

red {"Juan" : {"Perico" : "Andres"} } .
reduce in JSON : {"Juan" : {"Perico" : "Andres"}} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Map: {"Juan" : {"Perico" : "Andres"}}
Maude> 


In [23]:
result = %maude red {"Juan" : ["Perico", "Andres"]} .
print(result)    
assert 'Map: {"Juan" : ["Perico", "Andres"]}' in result 

red {"Juan" : ["Perico", "Andres"]} .
reduce in JSON : {"Juan" : ["Perico", "Andres"]} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Map: {"Juan" : ["Perico", "Andres"]}
Maude> 


In [24]:
result = %maude red \
{\
  "name" : "Manu Sporny",\
  "homepage" : "http://manu.sporny.org/",\
  "image" : "http://manu.sporny.org/images/manu.png"\
} .
print(result)
assert 'Map:' in result

red  {   "name" : "Manu Sporny",   "homepage" : "http://manu.sporny.org/",   "image" : "http://manu.sporny.org/images/manu.png" } .
reduce in JSON : {"name" : "Manu Sporny", "homepage" :
    "http://manu.sporny.org/", "image" :
    "http://manu.sporny.org/images/manu.png"} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Map: {"homepage" : "http://manu.sporny.org/", "image" :
    "http://manu.sporny.org/images/manu.png", "name" : "Manu Sporny"}
Maude> 


In [25]:
result = %maude red \
{\
  "@context" : {\
    "name" : "http://schema.org/name",\
    "image" : {\
      "@id" : "http://schema.org/image",\
      "@type" : "@id"\
    },\
    "homepage" : {\
      "@id" : "http://schema.org/url",\
      "@type" : "@id"\
    }\
  }\
} .
print(result)
assert 'Map:' in result

red  {   "@context" : {     "name" : "http://schema.org/name",     "image" : {       "@id" : "http://schema.org/image",       "@type" : "@id"     },     "homepage" : {       "@id" : "http://schema.org/url",       "@type" : "@id"     }   } } .
reduce in JSON : {"@context" : {"name" : "http://schema.org/name", "homepage" :
    {"@id" : "http://schema.org/url", "@type" : "@id"}, "image" : {"@id" :
    "http://schema.org/image", "@type" : "@id"}}} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Map: {"@context" : {"homepage" : {"@id" : "http://schema.org/url",
    "@type" : "@id"}, "image" : {"@id" : "http://schema.org/image", "@type" :
    "@id"}, "name" : "http://schema.org/name"}}
Maude> 


### IRIS
[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/#terms)

In [26]:
%%maude
fmod IRI is
protecting STRING .
sort IRI .
subsort IRI < String .
cmb S:String : IRI if find(S:String,":",0) :: NzNat .
endfm    


34 (lines sent before.)
--------------------------------------------
fmod IRI is
> protecting STRING .
> sort IRI .
> subsort IRI < String .
> cmb S:String : IRI if find(S:String,":",0) :: NzNat .
> endfm
Maude> 



In [27]:
result = %maude red "Http://perico" .
assert 'IRI: "Http://perico"' in result   

### Keywords

In [28]:
%%maude
fmod KEYWORD is
    protecting STRING .
    sort Keyword .
    subsort Keyword < String .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@included" : Keyword .
    mb "@graph"    : Keyword .
    mb "@nest"     : Keyword .
    mb "@type"     : Keyword .
    mb "@reverse"  : Keyword .
    mb "@index"    : Keyword .
endfm    


49 (lines sent before.)
--------------------------------------------
fmod KEYWORD is
> protecting STRING .
> sort Keyword .
> subsort Keyword < String .
> mb "@context"  : Keyword .
> mb "@id"       : Keyword .
> mb "@context"  : Keyword .
> mb "@id"       : Keyword .
> mb "@included" : Keyword .
> mb "@graph"    : Keyword .
> mb "@nest"     : Keyword .
> mb "@type"     : Keyword .
> mb "@reverse"  : Keyword .
> mb "@index"    : Keyword .
> endfm
Maude> 



In [29]:
result = %maude red "@context" . 
assert 'Keyword: "@context"' in result     

### Context

In [30]:
%%maude 
fmod CONTEXT is
    protecting IRI .
    extending JSON * ( sort Pair to TermDefinition,
                        sort Json to Value)  .
    sorts Context .
    
    subsort IRI Map < Value .    

    mb (string : value)  : TermDefinition .

    op {_} : TermDefinition* -> Context .
endfm  
        

61 (lines sent before.)
--------------------------------------------
fmod CONTEXT is
> protecting IRI .
> extending JSON * ( sort Pair to TermDefinition,
> sort Json to Value)  .
> sorts Context .
> 
> subsort IRI Map < Value .
> 
> mb (string : value)  : TermDefinition .
> 
> op {_} : TermDefinition* -> Context .
> endfm
Advisory: <automatic>: sort Value has been imported from both <standard input>,
    line 13 (fmod JSON-SORTS) and <standard input>, line 9 (fmod JSON-SORTS).
    TermDefinition*.
mb (string : value) : TermDefinition .
Maude> 



In [31]:
%%maude
--- Example 4: Context for the sample document in the previous section
red 
{
  "@context": {
    "name" : "http://schema.org/name",
    ---  This means that 'name' is shorthand for 'http://schema.org/name'
    "image" : {
      "@id" : "http://schema.org/image",
      ---  This means that 'image' is shorthand for 'http://schema.org/image'
      "@type" : "@id"
      --- This means that a string value associated with 'image'
      --- should be interpreted as an identifier that is an IRI
    },
    "homepage" : {
      "@id" : "http://schema.org/url",
      --- This means that 'homepage' is shorthand for 'http://schema.org/url'
      "@type" : "@id"
      ---  This means that a string value associated with 'homepage'
      ---  should be interpreted as an identifier that is an IRI 
    }
  }
} .

83 (lines sent before.)
--------------------------------------------
--- Example 4: Context for the sample document in the previous section
> red
> {
> "@context": {
> "name" : "http://schema.org/name",
> ---  This means that 'name' is shorthand for 'http://schema.org/name'
> "image" : {
> "@id" : "http://schema.org/image",
> ---  This means that 'image' is shorthand for 'http://schema.org/image'
> "@type" : "@id"
> --- This means that a string value associated with 'image'
> --- should be interpreted as an identifier that is an IRI
> },
> "homepage" : {
> "@id" : "http://schema.org/url",
> --- This means that 'homepage' is shorthand for 'http://schema.org/url'
> "@type" : "@id"
> ---  This means that a string value associated with 'homepage'
> ---  should be interpreted as an identifier that is an IRI
> }
> }
> } .
{ "@context": { <---*HERE*
Maude> 



In [32]:

fmod MERGE-CONTEXTS is
    protecting JSON * ( sort Pair to Map, sort Pair+ to Map+, sort Pair* to Map*, 
                      ) .    

    op merge : Map* Map* -> Map* .
    op merge : Context Context -> Context .

    eq merge(nul,context) = context .
    eq merge(context,nul) = context .
    eq merge(term : value ,  (term : value', map*))  = term : value', map*  .
    eq merge(term : value ,  map*  = term : value, map*  [owise] .
    eq merge({map,map*},{map*'}) = { merge(map,map*'),merge(map*,map*') } .     
fmod     

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' (1103232360.py, line 12)

In [ ]:
fmod EXPAND is
    protecting JSON * ( sort Map to Context ) .
    protecting IDENTIFIER .
    protecting KEYWORD .    
    sort IdOrKeyword .
    subsorts Identifier Keyword > IdOrKeyword .
    # most-recently-defined-wins mechanism.

    op apply-context : Context Node-Object -> Node-Object .
    op apply-context : Map* -> Map . 

    op merge-contexts : Context Context -> Context . 
        
    ce apply-context(Ctx,{ Map* }) = { apply-context(Ctx,Map*) } [owise] .
    ce apply-context(Ctx,{ "@context" : Ctx', Map* }) = { apply-context(merge-context(Ctx,Ctx)) , Map* } 
    
    
    

#### Terms

In [ ]:
%%maude
fmod TERM is
    protecting IRI .
    protecting Keyword .

    sort Identifier .
    subsort Iri < Identifier .
    sort IdOrKeyword .
    subsorts Identifier Keyword < IdOrKeyword .
        
    sort Term .
    subsort Term < String .
    op expand : String Context -> [URI] .
    cmb S:String : Term  if not S:String = "" 
                         /\ not S:String :: URI  
                         /\ not S:String :: Keyword 
                         /\ expand(S:String, Ctx:Context) :: IdOrKeyword .
                         /\ X:String :: Identifier or X:String :: Keyword .
endfm

### JSON-LD BASIC

In [ ]:
%%maude
fmod JSON-LD-BASIC is
    protecting JSON * (sort Pair to Entry, Pair+ to Entry+, Pair* to Entry*,
                       sort Json to Value) .

    sorts Term IRI Keyword .
    subsorts Term IRI Keyword < String .

    vars S : String . 
        
    cmb S : Keyword if substr(S,0,1) == "@" .
    cmb S : IRI     if find(S,":",0) =/= notFound .
    cmb S : Term    if find(S,":",0) == notFound /\ substr(S,0,1) =/= "@" .    

    --- The Context 
 
    sort Context_Map ,
    subsort Context_Map < Entry .    
                
    mb T:term : V:Value  : Context_Map .   
        
    sorts Context_Map+ Context_Map* .
    subsorts Context_Map < Context_Map+ < Context_Map* .
    subsort Context_Map  < Entry .    
    subsort Context_Map+ < Entry+ .    
    subsort Context_Map* < Entry* . 
    
    op _,_ : Context_Map* Context_Map* -> Context_Map* [ditto] .
    op _,_ : Context_Map* Context_Map+ -> Context_Map+ [ditto] .                    

    sort Context_Entry .
    subsort Context_Entry  < Entry .    
    
    vars P : Entry .
    cmb P : Context_Entry if "@context" : { CE*:context_entry+ } := P .  

    ---  Context semantics:    

    vars T   : Term .
    vars Iri : IRI .
    vars V   : Value . 
    vars ... : context_entry* .
        
    eq { "@context" : { T : Iri }, T : V, ...} = { "@context" : { T : Iri }, Iri : V, ...} .

    --- The Identity entry 
    
    sort Identity .
    subsort Identity < Entry .
    cmb P:Entry : Identity if "@id" : Iri:IRI := P .

    --- The Type entry

    sort Type-Entry .
    subsort Type-Entry < Entry .
    cmb P:Entry : Type-Entry if "@type" : Iri:IRI := P .

endfm        

In [ ]:
%%maude 
show modules .

In [ ]:
%%maude 
red "@context" .    
red "pepe" .    
red    

In [ ]:
%%maude 
--- Example 4
red 
 "@context" : {
    "name" : "http://schema.org/name",
    "image" : {
      "@id" : "http://schema.org/image",
      "@type" : "@id"
    }
} .

In [ ]:
%%maude
red {
  "@context" : {
    "name" : "http://schema.org/name"
  },
  "name" : "Manu Sporny",
  "status" : "trollin'"
} .

### HTML

## TO-DO
* Remove trailibg newlines grom cell.
* A command that ads a path to MAUDE_LIB

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()